In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
class2idx = {
    'no_relevant': 0,
    'relevant_minus': 1,
    'relevant': 2,
    'relevant_plus': 3,
}

idx2class = {v: k for k, v in class2idx.items()}

In [5]:
# posts = pd.read_parquet("/kaggle/input/russian-ai-olympiad-2024-b/items.parquet")

In [6]:
# itemId2idx = dict()
# for i, row in tqdm(posts.iterrows(), total=len(posts)):
#     itemId2idx[row['itemId']] = i

In [7]:
# train = pd.read_csv("/kaggle/input/russian-ai-olympiad-2024-b/train.csv")
# test = pd.read_csv("/kaggle/input/russian-ai-olympiad-2024-b/test.csv")
# subm = pd.read_csv("/kaggle/input/russian-ai-olympiad-2024-b/sample_submission.csv")

In [8]:
# train = train[~train['target'].isin(['doubles', 'unavailable'])]

In [9]:
# tqdm.pandas()

# def get_data_from_posts(row):
#     idx1, idx2 = itemId2idx[row['leftItemId']], itemId2idx[row['rightItemId']]
#     post1, post2 = posts.iloc[idx1], posts.iloc[idx2]

#     if row.get('target') is not None:
#         row['target'] = class2idx[row['target']]
        
    
#     row['text1'] = "Название: " + post1['title'] + "\n\nСодержание: " + post1['content']
    
#     row['text2'] = "Название: " + post2['title'] + "\n\nСодержание: " + post2['content']

#     row['same_author'] = 1 if post1['authorId'] == post2['authorId'] else 0
    
#     return row
    

# train = train.progress_apply(get_data_from_posts, axis=1)
# train.drop(columns=['leftItemId', 'rightItemId'], inplace=True)

# test = test.progress_apply(get_data_from_posts, axis=1)
# test.drop(columns=['leftItemId', 'rightItemId'], inplace=True)

In [10]:
train = pd.read_parquet("/kaggle/input/russian-ai-olympiad-2024-b/new_train.parquet")
test = pd.read_parquet("/kaggle/input/russian-ai-olympiad-2024-b/new_test.parquet")
subm = pd.read_csv("/kaggle/input/russian-ai-olympiad-2024-b/sample_submission.csv")

In [12]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train, test_size=0.05, stratify=train['target'], random_state=42)

train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

In [17]:
!pip install --upgrade protobuf==3.20.3 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 2.3 MB/s eta 0:00:00 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
pydrive2 1.21.3 requires cryptograph

In [18]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model_emb = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru").to(device)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors:   0%|          | 0.00/516M [00:00<?, ?B/s]

In [21]:
%%time

sentences = ["Hello World", "Привет Мир"]

encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=64, return_tensors='pt')
encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
with torch.no_grad():
    model_output = model_emb(**encoded_input)
embeddings = model_output.pooler_output
embeddings = torch.nn.functional.normalize(embeddings)
print(embeddings)

tensor([[-0.0714, -0.0772, -0.0305,  ...,  0.0136, -0.0402, -0.0245],
        [-0.0644, -0.0687, -0.0521,  ...,  0.0139, -0.0362, -0.0213]],
       device='cuda:0')
CPU times: user 15.2 ms, sys: 458 µs, total: 15.6 ms
Wall time: 13.7 ms


In [22]:
model_emb.device

device(type='cuda', index=0)

In [26]:
# def get_embeddings(df, batch_size=64):
#     emb1 = []
#     for i in tqdm(range(0, len(df), batch_size), desc='Getting text1 embeddings'):
#         encoded_input = tokenizer(df['text1'][i:i+batch_size].tolist(), padding=True, truncation=True, max_length=64, return_tensors='pt')
#         encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
#         with torch.no_grad():
#             model_output = model_emb(**encoded_input)
#         emb = model_output.pooler_output
#         emb = torch.nn.functional.normalize(emb)
#         emb1.append(emb.cpu())
#     emb1 = torch.cat(emb1)

#     emb2 = []
#     for i in tqdm(range(0, len(df), batch_size), desc='Getting text2 embeddings'):
#         encoded_input = tokenizer(df['text2'][i:i+batch_size].tolist(), padding=True, truncation=True, max_length=64, return_tensors='pt')
#         encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
#         with torch.no_grad():
#             model_output = model_emb(**encoded_input)
#         emb = model_output.pooler_output
#         emb = torch.nn.functional.normalize(emb)
#         emb2.append(emb.cpu())
#     emb2 = torch.cat(emb2)

#     embeddings = torch.cat([emb1, emb2], dim=1)
#     return embeddings

In [28]:
# train_embeddings = get_embeddings(train)
# valid_embeddings = get_embeddings(valid)

Getting text1 embeddings:   0%|          | 0/3095 [00:00<?, ?it/s]

Getting text2 embeddings:   0%|          | 0/3095 [00:00<?, ?it/s]

Getting text1 embeddings:   0%|          | 0/163 [00:00<?, ?it/s]

Getting text2 embeddings:   0%|          | 0/163 [00:00<?, ?it/s]

In [ ]:
# train_embeddings = torch.load('train_embeddings.pt')
# valid_embeddings = torch.load('valid_embeddings.pt')

In [70]:
class_counts = torch.tensor(train['target'].value_counts().sort_index().values, dtype=torch.float)
weights = 1.0 / class_counts  

weights = weights / weights.sum() * len(class_counts)  
weights

tensor([0.9734, 0.5328, 0.5495, 1.9443])

In [96]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

BATCH_SIZE = 64

# def get_embeddings(text):
#     encoded_input = tokenizer(text, padding=True, truncation=True, max_length=64, return_tensors='pt')
#     encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
#     with torch.no_grad():
#         model_output = model_emb(**encoded_input)
#     embeddings = model_output.pooler_output
#     embeddings = torch.nn.functional.normalize(embeddings)
#     return embeddings

class PostDataset(Dataset):
    def __init__(self, embeddings, target=None):
        self.embeddings = embeddings
        self.target = target
    
    def __len__(self):
        return len(self.embeddings)
    def __getitem__(self, idx):
        
        if self.target is not None:
            return self.embeddings[idx], torch.tensor(self.target[idx])
        return self.embeddings[idx]

train_ds = PostDataset(train_embeddings*10, train['target'])
valid_ds = PostDataset(valid_embeddings*10, valid['target'])

sample_weights_train = (1 / class_counts)[train_ds.target]
sample_weights_valid = (1 / class_counts)[valid_ds.target]

sampler_train = WeightedRandomSampler(sample_weights_train, num_samples=len(sample_weights_train), replacement=True)
sampler_valid = WeightedRandomSampler(sample_weights_valid, num_samples=len(sample_weights_valid), replacement=True)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler_train)
valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, sampler=sampler_valid)

In [121]:
class EmbeddingsClassifier(nn.Module):
    def __init__(self, input_size=1536, num_classes=4):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.layers(x)


epochs = 10
log_rate = 1

model = EmbeddingsClassifier()
model.to(device)

loss_fn = nn.CrossEntropyLoss(weight=weights.to(device))
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4, weight_decay=1e-4)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs*len(train_loader))

In [122]:
for epoch in tqdm(range(epochs), desc='Training'):
    model.train()
    running_train_loss = 0
    all_preds = []
    i = 0
    for X, y in (pbar := tqdm(train_loader, leave=False, desc='Train DataLoader')):
        i += 1
        X, y = X.to(device), y.to(device)
        with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
            logits = model(X)
            loss = loss_fn(logits, y)

        probs = torch.argmax(torch.softmax(logits, dim=-1), dim=-1)
        all_preds.extend(probs.detach().cpu().tolist())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()

        running_train_loss += loss.item()
        pbar.set_postfix({'loss': f"{running_train_loss/i:.5f}",}) 
                          # "diversity": torch.bincount(torch.tensor(all_preds)).tolist()})
        
    if (epoch+1)%log_rate==0:
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {running_train_loss/i:.5f}")

    model.eval()
    all_preds, all_targets = [], []
    i = 0
    for X, y in (pbar := tqdm(valid_loader, leave=False, desc='Valid DataLoader')):
        i += 1
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
                logits = model(X)
        probs = torch.argmax(torch.softmax(logits, dim=-1), dim=-1)
        all_preds.extend(probs.detach().cpu().tolist())
        all_targets.extend(y.cpu().tolist())
    
    f1 = f1_score(all_targets, all_preds, average='weighted')
    if (epoch+1)%log_rate==0:
        print(f"Weighted F1 Score: {f1:.5f}")
        print(f"Diversity: {torch.bincount(torch.tensor(all_preds)).tolist()}")

Training:   0%|          | 0/10 [00:00<?, ?it/s]

Train DataLoader:   0%|          | 0/3095 [00:00<?, ?it/s]

Epoch 1/10 | Train Loss: 1.07785


Valid DataLoader:   0%|          | 0/163 [00:00<?, ?it/s]

Weighted F1 Score: 0.30623
Diversity: [5080, 604, 40, 4701]


...

Weighted F1 Score: 0.43562
Diversity: [3031, 2010, 2125, 3259]


Train DataLoader:   0%|          | 0/3095 [00:00<?, ?it/s]

Epoch 10/10 | Train Loss: 0.57247


Valid DataLoader:   0%|          | 0/163 [00:00<?, ?it/s]

Weighted F1 Score: 0.45238
Diversity: [2980, 2425, 2420, 2600]


In [123]:
# torch.save(model.state_dict(), "checkpoint-2.pth")

In [124]:
# test_embeddings = get_embeddings(test)

In [125]:
test_embeddings = torch.load('test_embeddings.pt')

In [126]:
test_ds = PostDataset(test_embeddings*10)

test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

In [127]:
model.eval()
all_preds = []
i = 0
for X in (pbar := tqdm(test_loader, desc='Test DataLoader')):
    i += 1
    X = X.to(device)
    with torch.no_grad():
        with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
            logits = model(X)
    probs = torch.argmax(torch.softmax(logits, dim=-1), dim=-1)
    all_preds.extend(probs.detach().cpu().tolist())

Test DataLoader:   0%|          | 0/807 [00:00<?, ?it/s]

In [128]:
subm['target'] = list(map(idx2class.get, all_preds))
subm.to_csv("submission.csv", index=False)
subm

,Unnamed: 0,target
0,0,relevant
1,1,relevant_minus
2,2,relevant_plus
3,3,no_relevant
4,4,no_relevant
...,...,...
51631,51631,no_relevant
51632,51632,relevant_minus
51633,51633,relevant
51634,51634,relevant_plus
